In [14]:
%matplotlib inline
#https://gwemlightcurves.github.io/examples/index.html


import matplotlib.pyplot as plt
import h5py
import matplotlib

size=32
# lines.linewidth : 3
# lines.markersize : 10

#'grid.linestyle':'-',

#     'lines.linestyle':'-.',
ggplot_styles = {
    'axes.edgecolor': 'black',
    'axes.facecolor': 'white',#'whitesmoke',
    'axes.grid': True, # True
    'axes.grid.which': 'both',
    'grid.color': 'gainsboro',
    'grid.linewidth': '1.4',#'1.4'
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
    'font.size': size,
    'legend.fontsize':28,
    'text.usetex':True,
    'text.latex.preamble': r'\usepackage{amsmath}',
    'lines.linewidth':5,
    'font.weight':'bold',
    'axes.titlesize':size,
    'axes.labelsize':28,
    'xtick.labelsize':size,
    'xtick.labelsize':size,
    'xtick.color': 'black',#'555555',
    'xtick.major.bottom': True,
    'xtick.minor.bottom': True,#False
    'ytick.color': 'black',#'555555',
    'ytick.major.left': True,
    'ytick.minor.left': True,
    'axes.axisbelow':True
}

from os.path import isfile
from numpy import *
import glob
plt.rcParams.update(ggplot_styles)
#plt.rcParams['axes.axisbelow'] = True

import matplotlib.pyplot as plt


######
import numpy as np
import h5py
from scipy.interpolate import interp1d
from gwemlightcurves.EjectaFits.DiUj2017 import calc_meje, calc_vej

from gwemlightcurves.KNModels import KNTable
from gwemlightcurves import lightcurve_utils
from astropy.table import QTable
import pickle

import sys
from datetime import datetime


In [2]:
BASE_FOLDER = "/home/kanchan.soni/work/proj_ligo_india_winter/pycbc/gwemlightcurves/input/Monica/"
EOS = "sly"

EOS_M_R = np.loadtxt(BASE_FOLDER + f"/{EOS}_mr.dat", unpack=True)

radius_from_mass = interp1d(EOS_M_R[2], EOS_M_R[1])
mb_from_mass = interp1d(EOS_M_R[2], EOS_M_R[3])


In [3]:
injections = h5py.File("/home/kanchan.soni/work/proj_ligo_india_winter/pycbc/pycbc_live/make_injset/injections_500mpc.hdf", "r")


injection_file_indices_250 = '/home/kanchan.soni/work/proj_ligo_india_winter/pycbc/pycbc_live/pycbclive_runs/injections_idx_250.txt'

coinc_files = '/home/kanchan.soni/work/proj_ligo_india_winter/pycbc/pycbc_live/pycbclive_runs/injections_that_have_coincidences.txt'


In [4]:
inj_indices_250 = []

inj_index_for_coincs = []

with open(injection_file_indices_250, "r") as file_in:
    for line in file_in:
        inj_indices_250.append(int(line.split(':')[-1]))
        
with open(coinc_files,'r') as file_in:
    for line in file_in: 
        inj_index_for_coincs.append(int(line.split('/')[-2].split('inj')[-1]))
        
        
inj_indices_250 = np.asarray(inj_indices_250)
inj_index_for_coincs = np.asarray(inj_index_for_coincs)

In [5]:
print ('Total number of injections: ',len(inj_indices_250))
print ('Injections with coincidences:  ',len(inj_index_for_coincs))


Total number of injections:  250
Injections with coincidences:   204


In [9]:
redshift = 0

# subtracting 1 because of the counting starting from 0
m1 = injections["mass1"][:][inj_indices_250-1] / (1 + redshift)
m2 = injections["mass2"][:][inj_indices_250-1] / (1 + redshift)
mb1 = mb_from_mass(m1)
mb2 = mb_from_mass(m2)


s1z = injections["spin1z"][:][inj_indices_250-1] / (1 + redshift)
s2z = injections["spin2z"][:][inj_indices_250-1] / (1 + redshift)


distance = injections["distance"][:][inj_indices_250-1]


r1 = radius_from_mass(m1)
r2 = radius_from_mass(m2)
c1 = m1 / r1 * 1.476625
c2 = m2 / r2 * 1.476625
from astropy.table import QTable

In [10]:
t = QTable(
    [m1, m2, r1, r2, mb1, mb2, c1, c2, distance],
    names=["m1", "m2", "r1", "r2", "mb1", "mb2", "c1", "c2", "distance"],
)
tini = 0.1
tmax = 50.0
dt = 0.1
vmin = 0.02
th = 0.2
ph = 3.14
kappa = 1.0
eps = 1.58 * (10**10)
alp = 1.2
eth = 0.5
flgbct = 1
beta = 3.0
kappa_r = 1.0
slope_r = -1.2
theta_r = 0.0
Ye = 0.3

t["tini"] = tini
t["tmax"] = tmax
t["dt"] = dt
t["vmin"] = vmin
t["th"] = th
t["ph"] = ph
t["kappa"] = kappa
t["eps"] = eps
t["alp"] = alp
t["eth"] = eth
t["flgbct"] = flgbct
t["beta"] = beta
t["kappa_r"] = kappa_r
t["slope_r"] = slope_r
t["theta_r"] = theta_r
t["Ye"] = Ye


In [11]:
models = ["DiUj2017"]   
model_tables = {}

for model in models:
    model_tables[model] = KNTable.model(model, t)

for model in models:
    model_tables[model] = lightcurve_utils.calc_peak_mags(model_tables[model])
    model_tables[model] = lightcurve_utils.interpolate_mags_lbol(model_tables[model])


In [ ]:

# distance = 100 #Mpc
# plot = KNTable.plot_mag_panels(model_tables, distance=distance)
# plot.show()


# %%
plt.figure()
plt.plot(model_tables["DiUj2017"]["mag_z"][0])
plt.gca().invert_yaxis()
# plt.savefig("temp.jpg")
plt.show()

pickle.dump(model_table,open('bns_Bulla_parameter_grid_%sband.dat'%(filt),'wb'))